In [ ]:
import os
import numpy as np
import pandas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow import keras
import math
import seaborn as sns
from sqlalchemy import create_engine

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = '258080'
host = 'localhost'
port = 5432
database_name = 'churn_pred'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
connection = engine.connect()

In [ ]:
# Loading and displaying a dataset
df = pd.read_sql("SELECT * FROM fact_churn", connection)
df.drop(['area_code_area_code_415', 'area_code_area_code_510'], axis = 1, inplace = True)
df.head()

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
X = df.drop(['churn'], axis=1)
y = df['churn'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.25)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Checking data split in training
X_train.shape

In [ ]:
# Checking data split in testing
X_test.shape

In [ ]:
# Retrieving sample of the data
X.sample(5)

In [ ]:
# Converting the dataframe into a Tensorflow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train_scaled, label='tensorflow')

In [ ]:
# Tuning Hyper-parameter
tuner = tfdf.tuner.RandomSearch(num_trials=20)

In [ ]:
# Optimising Hyper-parameter
tuner.discret("max_depth", [4, 5, 6, 7])

In [ ]:
# Creating a Random Forest model
model = tfdf.keras.RandomForestModel(tuner=tuner)

In [ ]:
# Training the model
model.fit(train_ds)

In [ ]:
# Displaying the model summary
print(model.summary())

In [ ]:
# Evaluating the model (making predictions)
model.evaluate(test_ds)

In [ ]:
# Saving the model to a TensorFlow SaveModel to create an infrastructure, test and possibly deploy the decision forest
# model.save('customer-churn-prediction/RFC-model')